In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import os
os.chdir('/content/drive/MyDrive/딥러닝개론/Final Project')

# data

In [ ]:
rev_data = pd.read_csv('/content/drive/MyDrive/딥러닝개론/Final Project/data/rev_data_final.csv', index_col=0)
rev_data

,review,score,label
0,친절하시고 깔끔하고 좋았습니다,5,1
1,조용하고 고기도 굿,5,1
2,"갈비탕과 냉면, 육회비빔밥이 맛있습니다.",4,1
3,대체적으로 만족하나\n와인의 구성이 살짝 아쉬움,4,1
4,고기도 맛있고 서비스는 더 최고입니다~,5,1
...,...,...,...
8009,가짜원조... 줄서는사람 뭐지???,1,0
8010,짝퉁 여긴 원조랑 아무 관계 직원 연관되게 없는 그냥 이름만 .. 가짜입니다 속지 마세요,1,0
8011,가성비 별로 비싸기만함,1,0
8012,별 한개주기도 아까움. 별을 줘야 등록돼서 평가함! 나 같은 사람 또 안생겼으면 좋...,1,0


In [ ]:
shopping_data = pd.read_csv('/content/drive/MyDrive/딥러닝개론/Final Project/data/shopping_rev_final.csv', index_col=0)
shopping_data = shopping_data[:10000]
shopping_data

,review,score,label
0,배공빠르고 굿,5,1
1,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,2,0
2,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...,5,1
3,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,2,0
4,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,5,1
...,...,...,...
9995,기대를 너무 많이했나봐요 ........ 일단 머릿결 상하신 분은 고민좀 해보세요 ...,1,0
9996,너무 귀엽고 폭신폭신하고 예뻐요,5,1
9997,눈뒤에 실거는부분이커서 다뜨고 눈을붙일수가없네요 ㅜ,1,0
9998,짜증 옷이 뒤틀려서 왔음. 짜증 ㄷ신ㅅㅇㅌㄹ츤ㅅㄴㅅㅌㅅ가ㅡㅡㅎㄱㄴㅅ듣ㅅㄱᆞㄷ쑈는핟ㅎ...,2,0


In [ ]:
total_data = pd.concat([rev_data, shopping_data])
total_data = total_data.reset_index(drop=True)
total_data

,review,score,label
0,친절하시고 깔끔하고 좋았습니다,5,1
1,조용하고 고기도 굿,5,1
2,"갈비탕과 냉면, 육회비빔밥이 맛있습니다.",4,1
3,대체적으로 만족하나\n와인의 구성이 살짝 아쉬움,4,1
4,고기도 맛있고 서비스는 더 최고입니다~,5,1
...,...,...,...
18926,기대를 너무 많이했나봐요 ........ 일단 머릿결 상하신 분은 고민좀 해보세요 ...,1,0
18927,너무 귀엽고 폭신폭신하고 예뻐요,5,1
18928,눈뒤에 실거는부분이커서 다뜨고 눈을붙일수가없네요 ㅜ,1,0
18929,짜증 옷이 뒤틀려서 왔음. 짜증 ㄷ신ㅅㅇㅌㄹ츤ㅅㄴㅅㅌㅅ가ㅡㅡㅎㄱㄴㅅ듣ㅅㄱᆞㄷ쑈는핟ㅎ...,2,0


# KoGPT-2

## 이중분류 with shopping_rev_final.csv

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 15.6 MB/s 
     |████████████████████████████████| 182 kB 79.5 MB/s 
     |████████████████████████████████| 7.6 MB 52.4 MB/s 


In [ ]:
import transformers
transformers.__version__

'4.25.1'

In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import AutoTokenizer, TFGPT2Model
from tensorflow.keras.preprocessing.sequence import pad_sequences

### train & test split

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(total_data, test_size=0.2, random_state=0, stratify=total_data['label'])

In [ ]:
print('train_data shape is:', train_data.shape)
print('test_data shape is:', test_data.shape)

train_data shape is: (15144, 3)
test_data shape is: (3787, 3)


### tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### tokenizer 확인

In [ ]:
print(tokenizer.encode(train_data.iloc[0, 0]))

[13493, 12832, 8046, 8084]


In [ ]:
print(tokenizer.tokenize(train_data.iloc[0, 0]))

['▁기대', '▁이하', '예', '요']


In [ ]:
tokenizer.decode(tokenizer.encode(train_data.iloc[0, 0]))

'기대 이하예요'

In [ ]:
for elem in tokenizer.encode(train_data.iloc[0, 0]):
  print(tokenizer.decode(elem))

기대
이하
예
요


In [ ]:
encoded_result = tokenizer.encode(train_data.iloc[0, 0], max_length=128, pad_to_max_length=True, truncation=True)
print(encoded_result)
print('길이 :', len(encoded_result))

[13493, 12832, 8046, 8084, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
길이 : 128


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


#### tokenizer 설정

In [ ]:
max_seq_len = 128

In [ ]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

    input_ids, data_labels = [], []
    
    for example, label in tqdm(zip(examples, labels), total=len(examples)):

        bos_token = [tokenizer.bos_token]
        eos_token = [tokenizer.eos_token]
        tokens = bos_token + tokenizer.tokenize(example) + eos_token
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        input_ids.append(input_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return input_ids, data_labels

In [ ]:
train_X, train_y = convert_examples_to_features(train_data['review'], train_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 15144/15144 [00:10<00:00, 1481.43it/s]


In [ ]:
test_X, test_y = convert_examples_to_features(test_data['review'], test_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 3787/3787 [00:02<00:00, 1639.79it/s]


In [ ]:
# 최대 길이: 128
input_id = train_X[0]
label = train_y[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))
print('레이블 :',label)

단어에 대한 정수 인코딩 : [    1 13493 12832  8046  8084     1     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3]
각 인코딩의 길이 : 128
정수 인코딩 복원 : </s> 기대 이하예요</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

### model

In [ ]:
model = TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.11.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'lm_head.weight', 'transformer.h.7.attn.masked_bias', 'transformer.h.0.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [ ]:
input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
outputs = model([input_ids_layer])

In [ ]:
print(outputs)

TFBaseModelOutputWithPastAndCrossAttentions(last_hidden_state=<KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'tfgpt2_model')>, past_key_values=(<KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')

In [ ]:
print(outputs[0])

KerasTensor(type_spec=TensorSpec(shape=(None, 128, 768), dtype=tf.float32, name=None), name='tfgpt2_model/transformer/Reshape_2:0', description="created by layer 'tfgpt2_model'")


In [ ]:
class TFGPT2ForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name):
        super(TFGPT2ForSequenceClassification, self).__init__()
        self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt=True)
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.classifier = tf.keras.layers.Dense(1,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='sigmoid',
                                                name='classifier')

    def call(self, inputs):
        outputs = self.gpt(input_ids=inputs)
        cls_token = outputs[0][:, -1]
        cls_token = self.dropout(cls_token)
        prediction = self.classifier(cls_token)

        return prediction

#### TPU 사용시

In [ ]:
# TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [ ]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [ ]:
with strategy.scope():
  model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2")
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.BinaryCrossentropy()
  model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

#### GPU 사용시

In [ ]:
model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2")
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.0.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'lm_head.weight', 'transformer.h.8.attn.masked_bias', 'transformer.h.5.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

#### -

In [ ]:
model.fit(train_X, train_y, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
379/379 [==============================] - 350s 884ms/step - loss: 0.4831 - accuracy: 0.7922 - val_loss: 0.2716 - val_accuracy: 0.8940
Epoch 2/20
379/379 [==============================] - 333s 878ms/step - loss: 0.2626 - accuracy: 0.8982 - val_loss: 0.2642 - val_accuracy: 0.8950
Epoch 3/20
379/379 [==============================] - 333s 878ms/step - loss: 0.2025 - accuracy: 0.9227 - val_loss: 0.2571 - val_accuracy: 0.9082
Epoch 4/20
379/379 [==============================] - 333s 878ms/step - loss: 0.1662 - accuracy: 0.9392 - val_loss: 0.2660 - val_accuracy: 0.8996
Epoch 5/20
379/379 [==============================] - 333s 878ms/step - loss: 0.1257 - accuracy: 0.9549 - val_loss: 0.3047 - val_accuracy: 0.9046
Epoch 6/20
379/379 [==============================] - 333s 878ms/step - loss: 0.0963 - accuracy: 0.9648 - val_loss: 0.3134 - val_accuracy: 0.9003
Epoch 7/20
379/379 [==============================] - 333s 878ms/step - loss: 0.0855 - accuracy: 0.9700 - val_loss: 0.3391 -

In [ ]:
results = model.evaluate(test_X, test_y, batch_size=1024)
print("test loss, test acc: ", results)

4/4 [==============================] - 33s 8s/step - loss: 0.7344 - accuracy: 0.8767
test loss, test acc:  [0.7343599796295166, 0.8766834139823914]


In [ ]:
model.save_weights('/content/drive/MyDrive/딥러닝개론/Final Project/koGPT-2_shopo.h5')

### test

In [ ]:
model = TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.11.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'lm_head.weight', 'transformer.h.7.attn.masked_bias', 'transformer.h.0.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [ ]:
model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2")
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.11.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'lm_head.weight', 'transformer.h.7.attn.masked_bias', 'transformer.h.0.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [ ]:
model.fit(train_X, train_y, epochs=1, batch_size=32, validation_split=0.2)

379/379 [==============================] - 377s 926ms/step - loss: 0.5178 - accuracy: 0.7906 - val_loss: 0.3060 - val_accuracy: 0.8825


In [ ]:
model.load_weights('/content/drive/MyDrive/딥러닝개론/Final Project/koGPT-2_shopo.h5')

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

In [ ]:
results = model.evaluate(test_X, test_y, batch_size=1024)
print("test loss, test acc: ", results)

4/4 [==============================] - 42s 9s/step - loss: 0.7344 - accuracy: 0.8767
test loss, test acc:  [0.7343599796295166, 0.8766834139823914]


In [ ]:
def sentiment_predict(new_sentence):

  bos_token = [tokenizer.bos_token]
  eos_token = [tokenizer.eos_token]
  tokens = bos_token + tokenizer.tokenize(new_sentence) + eos_token
  input_id = tokenizer.convert_tokens_to_ids(tokens)
  input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]
  input_id = np.array([input_id])
  score = model.predict(input_id)[0][0]

  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

#### 시험용

In [ ]:
sentiment_predict('할로윈 분위기도 물씬 나고 제가 평소에 너무 좋아하는 우드톤과 아늑한 분위기라 너무 맘에 들었어요ㅠㅠ')

1/1 [==============================] - 0s 39ms/step
100.00% 확률로 긍정 리뷰입니다.



In [ ]:
sentiment_predict('그리고 사장님께서 너무 친절하고 귀여우시고 선곡 센스도 최고👍')

1/1 [==============================] - 0s 37ms/step
100.00% 확률로 긍정 리뷰입니다.



In [ ]:
sentiment_predict('저렴한 전집이지만 바로 부쳐주시니 맛은 보장됩니다.')

1/1 [==============================] - 0s 43ms/step
99.01% 확률로 긍정 리뷰입니다.



In [ ]:
sentiment_predict('조금씩 시켜먹기 좋게 메뉴판 구성이 되어있고 사이드도 다채롭습니다.')

1/1 [==============================] - 0s 34ms/step
99.99% 확률로 긍정 리뷰입니다.



In [ ]:
sentiment_predict('정문 근처 카페 찾다가 발견했는데 진짜 올해 간 카페 중에 제일 좋았어요...♥')

1/1 [==============================] - 0s 40ms/step
100.00% 확률로 긍정 리뷰입니다.



In [ ]:
sentiment_predict('크림치즈크로플 너무 맛있었습니다..!')

1/1 [==============================] - 0s 38ms/step
100.00% 확률로 긍정 리뷰입니다.



In [ ]:
sentiment_predict('카야크로플은 호불호 갈릴거 같은데 저는 괜찮았습니다.')

1/1 [==============================] - 0s 72ms/step
99.97% 확률로 긍정 리뷰입니다.



In [ ]:
sentiment_predict('콘센트도 많고 테이블 크기도 적당해서 노트북 작업하기 좋은데 다만 테이블 높이가 좀 낮아서 목이 아팠어요ㅠㅠ')

1/1 [==============================] - 0s 40ms/step
53.74% 확률로 긍정 리뷰입니다.



In [ ]:
sentiment_predict('엄마랑 방문했어요')

1/1 [==============================] - 0s 36ms/step
97.91% 확률로 긍정 리뷰입니다.



In [ ]:
sentiment_predict('존맛탱탱')

1/1 [==============================] - 0s 25ms/step
100.00% 확률로 긍정 리뷰입니다.



In [ ]:
sentiment_predict('개노맛')

1/1 [==============================] - 0s 24ms/step
56.89% 확률로 긍정 리뷰입니다.



In [ ]:
sentiment_predict('너무 별로에요 다신 안갈듯')

1/1 [==============================] - 0s 32ms/step
100.00% 확률로 부정 리뷰입니다.



In [ ]:
sentiment_predict('걍그래요')

1/1 [==============================] - 0s 23ms/step
100.00% 확률로 부정 리뷰입니다.



In [ ]:
sentiment_predict('맛없어요')

1/1 [==============================] - 0s 21ms/step
100.00% 확률로 부정 리뷰입니다.



In [ ]:
sentiment_predict('JMT')

1/1 [==============================] - 0s 21ms/step
99.83% 확률로 긍정 리뷰입니다.



In [ ]:
sentiment_predict('존맛탱구리')

1/1 [==============================] - 0s 29ms/step
100.00% 확률로 긍정 리뷰입니다.



In [ ]:
sentiment_predict('노맛')

1/1 [==============================] - 0s 25ms/step
99.70% 확률로 부정 리뷰입니다.



#### 크롤링 데이터 predict

In [ ]:
pred_data = pd.read_csv('/content/drive/MyDrive/딥러닝개론/Final Project/data/predict_split_final.csv', index_col=0)
pred_data

,식당 인덱스,리뷰,review_split
0,0,저렴한 전집이지만 바로 부쳐주시니 맛은 보장됩니다. 조금씩 시켜먹기 좋게 메뉴판 구...,"['저렴한 전집이지만 바로 부쳐주시니 맛은 보장됩니다.', '조금씩 시켜먹기 좋게 ..."
1,0,굿ㅇ,['굿ㅇ']
2,0,친절해요,['친절해요']
3,0,친절해요,['친절해요']
4,0,굿,['굿']
...,...,...,...
112435,1047,생활의 달인 방송후 다음날 갔는데.. 겨우 구입했어요!!\n진짜 쫀득쫀득하니 맛있네요^^,"['생활의 달인 방송후 다음날 갔는데..', '겨우 구입했어요!!', '\\n진짜 ..."
112436,1047,떡이 맛있어요~,['떡이 맛있어요~']
112437,1047,사장님도 너무친절하시고 떡도 너무 맛있어요 ㅎㅎ 아침용으로 하나씩 챙겨먹을때도있고 ...,"['사장님도 너무친절하시고 떡도 너무 맛있어요 ㅎㅎ', '아침용으로 하나씩 챙겨먹을..."
112438,1047,떡이 종류가 별루없네요.직접만드는거같아요\n가래떡샀는데 따뜻하네요~,"['떡이 종류가 별루없네요.', '직접만드는거같아요', '\\n가래떡샀는데 따뜻하네..."


In [ ]:
kogpt_shopo = pd.read_csv('/content/drive/MyDrive/딥러닝개론/Final Project/data/predict_split_final.csv', index_col=0)
del kogpt_shopo['리뷰']
kogpt_shopo.columns = ['index', 'review']
kogpt_shopo = kogpt_shopo.reindex(columns=['index', 'review', 'score'])
kogpt_shopo = kogpt_shopo[:6972]
kogpt_shopo

,index,review,score
0,0,"['저렴한 전집이지만 바로 부쳐주시니 맛은 보장됩니다.', '조금씩 시켜먹기 좋게 ...",NaN
1,0,['굿ㅇ'],NaN
2,0,['친절해요'],NaN
3,0,['친절해요'],NaN
4,0,['굿'],NaN
...,...,...,...
6967,50,['좋아요'],NaN
6968,50,['굿'],NaN
6969,50,['좋아요'],NaN
6970,50,['좋아요'],NaN


In [ ]:
def sentiment_predict_result(new_sentence):

  bos_token = [tokenizer.bos_token]
  eos_token = [tokenizer.eos_token]
  tokens = bos_token + tokenizer.tokenize(new_sentence) + eos_token
  input_id = tokenizer.convert_tokens_to_ids(tokens)
  input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]
  input_id = np.array([input_id])
  score = model.predict(input_id)[0][0]

  return score

In [ ]:
def cal_score(text):
  score = 0
  text_list = text[2:-2].split("', '")

  for i in text_list:
    score += sentiment_predict_result(i)

  score /= len(text_list)

  return score

In [ ]:
kogpt_shopo['score'] = kogpt_shopo['review'].apply(cal_score)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1/1 [==============================] - 0s 35ms/step


In [ ]:
kogpt_shopo.to_csv('/content/drive/MyDrive/딥러닝개론/Final Project/kogpt_shopo.csv', encoding='utf-8-sig')

In [ ]:
kogpt_shopo_result = kogpt_shopo.groupby('index').score.mean()
kogpt_shopo_result

index
0     0.809018
1     0.811971
2     0.791125
3     0.820605
4     0.729042
5     0.840451
6     0.858767
7     0.821570
8     0.669737
9     0.889841
10    0.793241
11    0.799062
12    0.701078
13    0.763305
14    0.718679
15    0.827240
16    0.820057
17    0.763770
18    0.809673
19    0.809154
20    0.840307
21    0.824005
22    0.846000
23    0.825530
24    0.680974
25    0.807091
26    0.978523
27    0.684513
28    0.896898
29    0.896484
30    0.893244
31    0.889281
32    0.615095
33    0.820284
34    0.805352
35    0.836628
36    0.742704
37    0.859567
38    0.874027
39    0.805124
40    0.886281
41    0.780491
42    0.784371
43    0.909839
44    0.251145
45    0.858189
46    0.771489
47    0.843726
48    0.937703
49    0.837607
50    0.862176
Name: score, dtype: float64

In [ ]:
kogpt_shopo_result.to_csv('/content/drive/MyDrive/딥러닝개론/Final Project/kogpt_shopo_final_result.csv', encoding='utf-8-sig')

In [ ]:
kogpt_shopo_final_result = kogpt_shopo_result * 5

In [ ]:
kogpt_shopo_final_result.to_csv('/content/drive/MyDrive/딥러닝개론/Final Project/kogpt_shopo_final_result_5.csv', encoding='utf-8-sig')